In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys, os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('image', cmap='Blues')
from slmsuite.holography.algorithms import Hologram
from slmsuite.hardware.slms.slm import SLM
from slmsuite.hardware.cameras.camera import Camera
from slmsuite.hardware.cameraslms import FourierSLM
from slmsuite.holography.algorithms import SpotHologram
import copy
from IPython.display import Image
import imageio

def plot(title=""):
    _, axs = plt.subplots(1, 3, figsize=(16,4))

    if slm.phase_correction is None:
        correction = 0 * slm.phase
    else:
        correction = np.mod(slm.phase_correction, 2*np.pi)

    axs[0].set_title("Phase Correction")
    axs[0].imshow(
        correction,
        vmin=0,
        vmax=2*np.pi,
        interpolation="none",
        cmap="twilight"
    )

    axs[1].set_title("Displayed Phase")
    axs[1].imshow(
        np.mod(slm.phase, 2*np.pi),
        vmin=0,
        vmax=2*np.pi,
        interpolation="none",
        cmap="twilight"
    )

    axs[2].set_title("Camera Result")
    axs[2].imshow(cam.get_image())

    plt.suptitle(title)

    plt.show()

In [ ]:
from slmsuite.hardware.slms.meadowlark import Meadowlark
Meadowlark.info(verbose=True)
from slmsuite.hardware.cameras.thorlabs import ThorCam
ThorCam.info(verbose=True)

In [ ]:
# Fill with own camera info from above cell
slm = Meadowlark(slm_number=1, display_number=2, wav_um=.633, settle_time_s=.3)
cam = ThorCam(serial="02C5V", verbose=True)
fs = FourierSLM(cam, slm)


In [ ]:
## Now create the wavefront calibration file
# Step 1: Calibrate the Camera Fourier domain

cam.set_exposure(.05)
fs.fourier_calibrate(
    array_shape=[30, 20],           # Size of the calibration grid (Nx, Ny) [knm]
    array_pitch=[30, 40],           # Pitch of the calibration grid (x, y) [knm]
    plot=True
)

In [ ]:
# Step 2: Test a single superpixel
cam.set_exposure(.2)
movie = fs.wavefront_calibrate(
    interference_point=(900, 400),
    field_point=(.25, 0),
    field_point_units="freq",
    superpixel_size=50,
    test_superpixel=(16, 16),           # Testing mode
    autoexposure=False,
    plot=3                              # Special mode to generate a phase .gif
)

# Generate a phase .gif
imageio.mimsave('wavefront.gif', movie)
Image(filename="wavefront.gif")

In [ ]:
fs.wavefront_calibrate(
    interference_point=(1000, 700),
    field_point=(.25, 0),
    field_point_units="freq",
    superpixel_size=50,
    autoexposure=False
)

fs.save_wavefront_calibration()


In [ ]:
fs.process_wavefront_calibration(r2_threshold=.9, smooth=True, plot=True)


In [ ]:

fs.load_wavefront_calibration(plot=True);  